In [1]:
import numpy as np
import csv

from descriptors_values import DescriptorsValues

# Algorithm for ACC-transformation of peptides

### 1. Introduction

Proteins are large biomolecules and macromolecules that comprise one or more long chains of amino acid residues. Proteins perform a vast array of functions within organisms, including catalyzing metabolic reactions, DNA replication, responding to stimuli, providing structure to cells and organisms, and transporting molecules from one location to another.

These protein functions are of particular interest in various biology-related sciences like toxicology, pharmacology, immunology etc. If we want to assess hypothetical function of one given protein, then the approach is pretty straightforward - we perform a biological assay. However, in most cases we have a set containing hundreds or thousands of protein structures. We, of course cannot perform such number of experiments and thus an alternative approach is required. Might the best such approach is to do a computer modeling of a particular function. Those type of approaches in biology are known by the term **_in silico_**. An example of this is bacterial immunogenicity prediction by machine learning methods done by Dimitrov et al., 2020 [1].

In order to model any biological properties such as immunogenicity or allergenicity, the protein structures should be represented as meaningful number or set of numbers which carry information of its biological properties. The fore mentioned numbers are knows as molecular descriptors. Encoded that way, various machine learning algorithms can be applied on the protein sequence to model the desired biologic features. However, proteins consist of hundreds or thousands amino acids and that large variance of length would lead to a different number of features for each protein structure. A way to avoid this is to apply auto- and cross covariance (ACC) transformation (Wold et al., 1993 [2]) on the descriptors-encoded protein sequence, which is the main topic of this present work.  

### 2. Aims and tasks

The library **Biopython** has reliable methods for working with proteins in FASTA format. However, in many cases we have to deal with fragments of protein structures, called **peptides** in plain text format.

The aim of the project is to develop an algorithm for ACC-transformation of peptides. This has been achieved through the following steps:<br>$\;\;\;\;\;\;$1. Read a .txt file with peptides in plain text format.<br>$\;\;\;\;\;\;$2. Encode the peptides with descriptors<br>$\;\;\;\;\;\;$3. Apply ACC-transformation to the descriptors-encoded sequences.<br>$\;\;\;\;\;\;$4. Write the results in a .csv file.

### 3. Materials and methods

#### 3.1. Preprocessing

Before we start with the substantial work, we need to define some global variables which will find their usage later:

In [2]:
file_to_transform = "demo_dataset.txt"  # The .txt file with peptides in plain text format

labels = []  # Labels for the .csv results-containing file

lag = 7  # Can vary depending on the design of experiment

descriptors_number = 5  # Depends on the type of the used descriptors

descriptor_matrix = {}  # Dictionary with key = peptide and value = its descriptor-encoded amino acids

acc_values = []  # 2D-array containing the ACC-transformed peptides

Now, we have to create appropriate `labels` for the .csv results-containing file. First column will contain the peptide, last column will contain the class to which every peptide belongs and between them will be the ACC-values:

In [3]:
def create_fields():
    
    for lag_value in range(1, lag+1):
        
        for first_amino_acid_index in range(1, descriptors_number+1):
            
            for second_amino_acid_index in range(1, descriptors_number+1):
                labels.append(f"ACC{first_amino_acid_index}{second_amino_acid_index}{lag_value}")
                
    labels.insert(0, "Peptide")
    labels.insert(len(labels), "Class")

#### 3.2. Read peptides

Having done the preprocessing, now we can read the `.txt` file which contains the peptides in plain text format. It contains only three peptides, but they will be completely enough for our purposes. The three peptides are **in vivo** confirmed human tumor epitopes (source - IEDB [3]). We have to store the peptides in the below defined local variable `peptides_list`. We also have to remove whitespace and new line characters (if present), otherwise later descriptors-encoding would raise exceptions:

In [4]:
def read_peptide():
    
    peptides_list = []
    
    lines = open(file_to_transform).readlines()
    
    for line in lines:
        line = line.replace('\n', '')
        line = line.replace(' ', '')
        peptides_list.append(line)
        
    return peptides_list

#### 3.3. Encode peptides with descriptors

Having read and stored the peptides, we can now process to encoding them with set of descriptors. Here we will use the $E$-descriptors which were proposed by Venkatarajan and Braun [4]. They derived five numerical values for each of the 20 naturally occurring amino acids based on the principal component analysis (PCA) of 237 physicochemical properties. They are represented in the `descriptors_values.py` file as a dictionary with `key` = amino acid (as a one letter code) and `value` = its corresponding $E$-descriptor. Since $E$-descriptors are 5 different types for each amino acid, we need to set the value of `descriptors_number` global variable to `5`. The `descriptors_values.py` file contains also the $Z$-descriptors (Hellberg et al., 1987 [5]). In order to use them instead, we need to set the value of `descriptors_number` global variable to `3`. 
If a record is not a valid sequence (one letter code-encoded peptide in plain text format) a proper user-friendly exception message should be raised and only valid records have to be processed.

In [5]:
def encode_peptides_with_descriptors():
    
    for peptide in read_peptide():
        descriptor_matrix[peptide] = []
        
        try:
            for amino_acid in peptide:
                descriptor_matrix[peptide].append(DescriptorsValues.E_DESCRIPTORS_DICT[amino_acid])
        except:
            print(f"'{peptide}' record is invalid - not a one letter-code encode peptide in a plain text format")
            del descriptor_matrix[peptide]

#### 3.4. ACC-transformation

So far we have created a `dictionary` with `key` = peptide and `value` = 2-dimensional list containing a set of 5 descriptor values for each amino acid in a peptide sequence. Now we can perform the ACC-transformation for each peptide. The ACC (auto- and cross-covariance) transformation is given by the following formula:

$${ACC}_{j,k}(lag) = \sum_{i}^{n-lag} \frac{{E_{j,i}\times}E_{k,i+lag}}{n-lag}$$

Where:<br>$\;\;\;\;\;\;j, k$ - $E$-descriptors<br>$\;\;\;\;\;\;n$ - number of the amino acids in the
peptide<br>$\;\;\;\;\;\;i$ - index of an amino acid in the peptide ($i$ = 1, 2, 3, ..., $n$)<br>$\;\;\;\;\;\;lag$ - lag value

An intuitive approach of computing the ACC-values will be to loop through the 2-dimensional list for each peptide. This will require the use of several nested `for` loops and might not be computationally efficient for large datasets. In order to overcome this problem, linear algebra comes to a rescue. `Python` library `NumPy` has several built-in methods for working with linear structures, including matrix multiplication. All we have to do is to convert a python list to `numpy.array`.

In [6]:
def matrix_multiplication():
    
    for peptide, matrix in descriptor_matrix.items():
        record = [peptide]
        length = len(matrix)
        
        for lag_value in range(1, lag + 1):
            np_array = np.array(matrix)
            first_matrix = (np_array[:length - lag_value, :length]).transpose()
            second_matrix = np_array[0 + lag_value:, :length]
            result = np.matmul(first_matrix, second_matrix)
            
            for x in result:
                
                for y in x:
                    acc_value = y / (length - lag_value)
                    record.append(acc_value)
                    
        # '1' for immunogens, '0' for non-immunogens; since the three peptides in our dataset are immunogenic, we write '1'
        record.append(1)
        acc_values.append(record)

#### 3.5. Create results file

We have now applied the ACC-transformation for every peptide in our `demo_database.txt` dataset. Final step is to write the results in a `.csv` file:

In [7]:
def create_csv_file():
    
    with open('demo_database_results.csv', 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(labels)
        write.writerows(acc_values)

#### 3.6. Run project

Now we can wrap up our code and execute it:<br>$\;\;\;\;\;\;$<font color='red'>Do not forget to execute all the cells in their corresponding order before executing this last cell!</font>

In [8]:
create_fields()
read_peptide()
encode_peptides_with_descriptors()
matrix_multiplication()
create_csv_file()

### 4. Future perspectives

Having quantitative representation of peptide structures as a result of the now-developed ACC-transforming algorithm, a various machine learning methods (Random forest, XGBoost, Support vector machine etc.) could be applied in order to generalize certain biological functions (in our case - tumor immunogenicity). However, a much larger data set will be required in pair with a negative set of peptides which does not induce immunogenicity. 